In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Github/dus_mm/')
import numpy as np
import pandas as pd
import utils
import glob

from matplotlib import pyplot as plt
import seaborn as sb
os.chdir('data')

columnsX = ['freq','AX','BX','BY','CY','CX','DX','DY','AY']
columnsY = ['AX','BX','BY','CY','CX','DX','DY','AY','AQ','AL','BQ','BL','CQ','CL','DQ','DL']
columnsYout = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']

Mounted at /content/drive


In [3]:
X = pd.DataFrame()
y = pd.DataFrame()
numb_files = len(glob.glob('data_output*'))
for i in glob.glob('data_output*'): 
    numb = i[11]
    if (numb == 'data_output.txt'):
        numb = ''
    y = pd.concat([y,pd.read_table(os.path.join('data_output%s.txt' % numb), header = 0, names = columnsY, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float')], ignore_index = 1)
    X = pd.concat([X,pd.read_table(os.path.join('data_input%s.txt' % numb), header = 0, names = columnsX, 
                                   index_col = False, sep='\s+', engine='python', dtype ="float")], ignore_index = 1)
X = X.loc[:20*4+3]
y = y.loc[:20]
X = utils.init_in(X)
y = utils.decomposition(y)

In [130]:
class Layer():
  def __init__(self, neurons: int = 256, activation: str = 'linear'):
      self.neurons = neurons
      self.activation = activation

In [193]:
class NN_model():
  
  def __init__(self, input_dimension: int):
      self.sequential = [Layer(input_dimension)]
      self.theta = []
      self.loss = 'mean_squared_error'
      self.optimizer = 'adam'
      self.num_iterations = 10
      self.beta_1 = 0.9
      self.beta_2 = 0.9
      self.step_size = 0.1

  def add(self, layer: Layer):
      self.sequential.append(layer)
      self.initial_theta()

  def initial_theta(self):
      layer_in = self.sequential[-2]
      layer_out = self.sequential[-1]
      epsilon = np.sqrt(6)/np.sqrt(layer_in.neurons + layer_out.neurons)
      init_theta = np.random.rand(layer_out.neurons,layer_in.neurons + 1)*2*epsilon - epsilon
      self.theta.append(init_theta.reshape(1,np.size(init_theta)))

  def activation(self, layer: Layer, x, diff: bool = False):
      if (layer.activation == 'relu'):
        if (diff == True):
          x[x >= 0] = 1
          x[x < 0] = 0
        else:
          x[x < 0] = 0
      elif (layer.activation == 'linear'):
        pass
      return x

  def feedforward(self, x):
    a = x
    a_out = [a.reshape([1, a.size])]
    for i in range(1,len(self.sequential)):
      layer1 = self.sequential[i-1]
      layer2 = self.sequential[i]
      theta12 = self.theta[i-1].reshape(layer2.neurons, layer1.neurons+1)
      z = np.dot(np.concatenate((np.ones([a.shape[0],1]),a),axis = 1), theta12.T)
      a = self.activation(layer2, z)
      a_out.append(a.reshape([1, a.size]))
    return a_out

  def backprop(self, a_out, y): 
    grad_out = []
    for i in range(1,len(self.sequential)):
      layer1 = self.sequential[-(i+1)]
      layer2 = self.sequential[-i]
      a1 = a_out[-(i+1)].reshape(y.shape[0], layer1.neurons)
      a1 = np.concatenate((np.ones([a1.shape[0],1]),a1),axis = 1)
      a2 = a_out[-i].reshape(y.shape[0], layer2.neurons)
      if (i == 1):
        delta = self.compute_delta_end(a2, y)
      else:
        delta = np.dot(delta,theta12[:,1:]) *  self.activation(layer2, a2)
      grad = (1/y.shape[0])*(np.dot(delta.T, a1))
      grad_out.insert(0,grad.reshape(1,grad.size))
      theta12 = self.theta[-i].reshape(layer2.neurons, layer1.neurons+1)
    return grad_out

  def compute_delta_end(self,a_end,y):
    if (self.loss == 'mean_squared_error'):
      delta = (a_end - y)
    return delta 
  #def compute_loss(self, a_out):

  def optimize(self, x, y):
    if (self.optimizer == 'adam'):
      for t in range(self.num_iterations):
        a_out = dus_mm.feedforward(x)
        grad = dus_mm.backprop(a_out,y)
        m = self.beta_1 * m + (1 - self.beta_1) * grad
        v = beta_2 * v + (1 - beta_2) * np.power(grad, 2)
        m_hat = m / (1 - np.power(beta_1, t))
        v_hat = v / (1 - np.power(beta_2, t))
        w = w - step_size * m_hat / (np.sqrt(v_hat) + epsilon)

  def __repr__(self):
    return str(len(self.sequential))


In [194]:
dus_mm = NN_model(X.shape[1])
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(y.shape[1], 'linear'))

In [195]:
a_out = dus_mm.feedforward(X)
grad =dus_mm.backprop(a_out,y)

In [ ]:
X1 = X
X1 = np.concatenate((np.ones([X1.shape[0],1]),X1),axis = 1)
X2 = relu(np.dot(X1,theta1.T))
X2 = np.concatenate((np.ones([X2.shape[0],1]),X2),axis = 1)
X3 = relu(np.dot(X2,theta2.T))
X3 = np.concatenate((np.ones([X3.shape[0],1]),X3),axis = 1)
X4 = linear(np.dot(X3,theta3.T))

delta4 = (X4 - y)
delta3 = np.dot(delta4,theta3[:,1:]) * relu(X3[:,1:],True)
delta2 = np.dot(delta3,theta2[:,1:]) * relu(X2[:,1:],True)
grad1 =  (1/n)*(np.dot(delta2.T,X1))
grad2 =  (1/n)*(np.dot(delta3.T,X2))
grad3 =  (1/n)*(np.dot(delta4.T,X3))


In [ ]:
for t in range(num_iterations):
    grad = compute_gradient(x, y)
    m = beta_1 * m + (1 - beta_1) * grad
    v = beta_2 * v + (1 - beta_2) * np.power(grad, 2)
    m_hat = m / (1 - np.power(beta_1, t))
    v_hat = v / (1 - np.power(beta_2, t))
    w = w - step_size * m_hat / (np.sqrt(v_hat) + epsilon)